<a href="https://colab.research.google.com/github/Vishakh2012/finetuning/blob/main/sarsasm_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_json("/content/drive/MyDrive/Sarcasm_Headlines_Dataset.json")
df = df[['headline','is_sarcastic']]


In [ ]:
!pip install transformers


In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
df.shape

(26709, 2)

In [ ]:
import tensorflow as tf
buffer_size = 64

is_train = np.random.uniform(size = len(df))<0.8

train_raw = (tf.data.Dataset.
             from_tensor_slices((dict(tokenizer(list(df['headline'][is_train]), padding = True, truncation = True)),np.array(df['is_sarcastic'])[is_train ])).
             shuffle(len(df)).
             batch(64,drop_remainder = True)
            )

train_raw.prefetch(1)

test_raw = (tf.data.Dataset.
            from_tensor_slices((dict(tokenizer(list(df['headline'][~is_train]), padding = True, truncation = True)),np.array(df['is_sarcastic'])[~is_train ])).
            shuffle(len(df)).
            batch(64, drop_remainder = True)
            )


test_raw.prefetch(1)

<_PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(64, 56), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(64, 56), dtype=tf.int32, name=None)}, TensorSpec(shape=(64,), dtype=tf.int64, name=None))>

In [ ]:
train_raw

<_BatchDataset element_spec=({'input_ids': TensorSpec(shape=(64, 66), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(64, 66), dtype=tf.int32, name=None)}, TensorSpec(shape=(64,), dtype=tf.int64, name=None))>

In [ ]:
for item in train_raw.take(1):
    print(item)





({'input_ids': <tf.Tensor: shape=(64, 66), dtype=int32, numpy=
array([[ 101, 3492, 2210, ...,    0,    0,    0],
       [ 101, 8554, 3065, ...,    0,    0,    0],
       [ 101, 2327, 2702, ...,    0,    0,    0],
       ...,
       [ 101, 5956, 1011, ...,    0,    0,    0],
       [ 101, 8958, 2730, ...,    0,    0,    0],
       [ 101, 2013, 2369, ...,    0,    0,    0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(64, 66), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>}, <tf.Tensor: shape=(64,), dtype=int64, numpy=
array([0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0])>)


In [ ]:
# from transformers import create_optimizer
# batch_size = 64

# num_epochs = 5

# batches_per_epoch = len(train_raw) // batch_size

# total_train_steps = int(batches_per_epoch * num_epochs)

# optimizer, lr_schedule = create_optimizer(
#     init_lr=2e-5,
#     num_train_steps=total_train_steps,
#     weight_decay_rate=0.01,
#     num_warmup_steps=500,
# )

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from transformers import TFDistilBertForSequenceClassification
num_epochs = 3
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
adam = Adam()

model.compile(
    optimizer=Adam(5e-5),
    metrics=["accuracy"],
    loss = SparseCategoricalCrossentropy(from_logits = True)
)
model.fit(
    train_raw,
    validation_data=test_raw,
    epochs = num_epochs
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/3
333/333 [==============================] - 162s 408ms/step - loss: 0.2721 - accuracy: 0.8849 - val_loss: 0.2021 - val_accuracy: 0.9179
Epoch 2/3
333/333 [==============================] - 137s 412ms/step - loss: 0.1087 - accuracy: 0.9610 - val_loss: 0.2522 - val_accuracy: 0.9081
Epoch 3/3
333/333 [==============================] - 138s 415ms/step - loss: 0.0446 - accuracy: 0.9840 - val_loss: 0.2738 - val_accuracy: 0.9196


In [ ]:
model.evaluate(test_raw)

83/83 [==============================] - 9s 108ms/step - loss: 0.2753 - accuracy: 0.9191


[0.27532655000686646, 0.9190512299537659]

In [ ]:
text = "man walks on his hand going to work"
text_tok = dict(tokenizer(text , truncation = True, padding = True, return_tensors = 'tf' ))
prediction = model.predict(text_tok)

logits = prediction.logits

probabilities = tf.nn.softmax(logits, axis = 1)
print(probabilities)
class_probability = probabilities[0]
clp = class_probability.numpy()
print(class_probability.numpy())

if clp[0] > clp[1]:
  print("this line is not sarcastic")
else:
  print("this line is sarcastic")


1/1 [==============================] - 0s 33ms/step
tf.Tensor([[6.7046360e-04 9.9932957e-01]], shape=(1, 2), dtype=float32)
[6.7046360e-04 9.9932957e-01]
this line is sarcastic
